## Import necessary packages

In [1]:
import ccxt
import pandas as pd

## Initialize parameters and objects

In [2]:
# Initialize the CCXT exchange object
exchange = ccxt.binance()

# Fetch all available trading pairs on Binance
symbols = exchange.fetch_markets()
symbol_names = [symbol['symbol'] for symbol in symbols]

# Filter symbols based on USDT or USD
filtered_symbols = [symbol for symbol in symbol_names if 'USDT' in symbol]

# Define the timeframe for OHLCV (Open, High, Low, Close, Volume) data
timeframe = '1d'

## Fetch historical OHLCV data

In [ ]:
def fetch_ohlcv(symbol, timeframe, limit=100):
    # Fetch historical OHLCV data
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit + 20)  # Fetch additional 20 candles
    
    # Convert the data to a Pandas DataFrame
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    
    # Convert timestamp to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    
    return df.iloc[-limit:]  # Return only the last 'limit' candles

## detecting big move in market

In [4]:
def detect_big_move(df, volume_multiplier=3):
    # Calculate the average volume of the previous 20 candles
    df['avg_volume'] = df['volume'].rolling(window=20).mean()
    
    # Filter rows with volume 3 times greater than the average volume of the previous 20 candles
    big_moves = df[df['volume'] > volume_multiplier * df['avg_volume']]
    
    return big_moves

## Main script

In [ ]:
for symbol in symbol_names:
    try:
        # Fetch historical data
        df = fetch_ohlcv(symbol, timeframe, limit=20)
        
        # Detect big moves
        big_moves = detect_big_move(df, volume_multiplier=3)
        
        # Print the results
        if not big_moves.empty:
            print(f"Symbol: {symbol}")
            print(big_moves[['timestamp', 'close', 'volume', 'avg_volume']])
            print("\n" + "=" * 40 + "\n")
    except ccxt.NetworkError as e:
        print(f"Network error for {symbol}: {e}")
    except ccxt.ExchangeError as e:
        print(f"Exchange error for {symbol}: {e}")
    except Exception as e:
        print(f"An unexpected error occurred for {symbol}: {e}")
